# Import Dependencies

In [73]:
import mesa
import numpy as np
import math


import matplotlib.pyplot as plt
%matplotlib inline

# Helper Functions

In [74]:
def get_distance(pos_1, pos_2):
    '''
    Calculate the Euclidean distance between two positions

    used in trade.move()
    '''
    x1, y1 = pos_1
    x2, y2 = pos_2
    dx = x1 - x2
    dy = y1 - y2
    return math.sqrt(dx**2 + dy**2)

# Resource Classes

In [75]:
class Sugar(mesa.Agent):
    '''
    Sugar:
    - contains an amount of sugar
    - grows one amount of sugar each turn
    '''
    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(model, unique_id)
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar

    def step(self):
        '''
        Sugar Growth Function, Adds one unit of sugar each step until
        we reach max amount of sugar
        '''
        self.amount = min([self.max_sugar, self.amount + 1]) # Either max amount or add one, whichever is less

In [76]:
class Spice(mesa.Agent):
    '''
    Spice:
    - contains an amount of spice
    - grows 1 amount of spice each turn
    '''
    def __init__(self, unique_id, model, pos, max_spice):
        super().__init__(model, unique_id)
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self):
        '''
        Spice Growth Function, adds one unit of spice 
        '''
        self.amount = min([self.max_spice, self.amount+1])
        # print(self.unique_id, self.max_spice, self.amount)


# Trader Class

In [77]:
class Trader(mesa.Agent):
    '''
    Trader:
    - has a metabolism for sugar and spice
    - harvests and trades sugar and spice to survive and thrive
    '''
    def __init__(self, unique_id, model, pos, moore=False, sugar=0, spice=0, metabolism_sugar=0, metabolism_spice=0, vision=0):
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore # Which grid are they looking at - could learn more about
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision

    def get_sugar(self, pos):
        '''
        Used in self.get_sugar_amount()
        '''
        this_cell = self.model.grid.get_cell_list_contents(pos)
        for agent in this_cell:
            if type(agent) is Sugar:
                return agent
        return None

    def get_sugar_amount(self, pos):
        '''
        Used in self.move() as part of self.calculate_welfare()
        '''
        sugar_patch = self.get_sugar(pos)
        if sugar_patch:
            return sugar_patch.amount
        return 0
    
    def get_spice(self, pos):
        '''
        Used in self.get_spice_amount
        '''
        this_cell = self.model.grid.get_cell_list_contents(pos)
        for agent in this_cell:
            if type(agent) is Spice:
                return agent
        return None
    
    def get_spice_amount(self, pos):
        '''
        Used in self.move() as part of self.calculate_welfare()
        '''
        spice_patch = self.get_spice(pos)
        if spice_patch:
            return spice_patch.amount
        return 0

    def is_occupied_by_other(self, pos):
        '''
        Helper function for 1. Identify every possible move an agent could make
        '''
        if pos == self.pos:
            # Agents position is considered unoccupied as agent can stay there
            return False
        
        # Get the contents of each cell in neighborhood
        this_cell = self.model.grid.get_cell_list_contents(pos)
        for a in this_cell:
            # See if occupied by another agent
            if isinstance(a, Trader):
                return True
        return False
    
    def calculate_welfare(self, sugar, spice):
        '''
        Helper function 2. Maximize welfare for an agent
        '''
        # Calculate total resources agent has
        m_total = self.metabolism_sugar + self.metabolism_spice
        
        # Cobb-Douglas Function
        return sugar**(self.metabolism_sugar/m_total) * spice**(self.metabolism_spice/m_total)


    # Build the move function for the Trader
    def move(self):
        '''
        Function for trader agent to identify the optimal move location for each step in 4 parts:
            1. Identify every possible move an agent could make
            2. Determine which move maximizes welfare for the agent
            3. Find closest and best option
            4. Move to that option
        '''
        # 1. Identify all possible moves an agent could make
        neighbors = [i
                     for i in self.model.grid.get_neighborhood( # Get neighborhood from the grid that we created (50x50 sugar and spice scape)
                         self.pos, self.moore, True, self.vision # True refers to set center, set all things to what we want
                     ) if not self.is_occupied_by_other(i)]
        
        # 2. Determine which move maximizes the welfare for the agent
        welfares = [
            self.calculate_welfare(self.sugar + self.get_sugar_amount(pos), self.spice + self.get_spice_amount(pos)) for pos in neighbors
        ]

        # 3. Find the closest best option
        # Get the highest welfare in welfares
        max_welfare = max(welfares)

        # Get the index of max welfare cells - potential places to move
        candidate_indices = [i for i in range(len(welfares))
                             if math.isclose(welfares[i], max_welfare, rel_tol=1e-02)]
        
        # Convert index to positions of those cells, where the agent can move
        candidates = [neighbors[i] for i in candidate_indices]

        min_dist = min(get_distance(self.pos, pos) for pos in candidates)

        final_candidates = [pos for pos in candidates if math.isclose(get_distance(self.pos, pos), min_dist, rel_tol=1e-02)]

        self.random.shuffle(final_candidates)

        # 4. Finally, actually move the agent
        self.model.grid.move_agent(self, final_candidates[0])

        print(min_dist, final_candidates)

        # print(welfares)
        # print(self.pos, neighbors)
        

# Model Class

In [78]:
class SugarscapeG1mt(mesa.Model):
    '''
    A model class to manage Sugarscape with Traders (G1mt)
    from Growing Artificial Societies by Axtell and Epstein
    '''
    def __init__(self, 
                 width=50, 
                 height=50, 
                 initial_population=200, # All new parameters according to the Growing Artificial Societies
                 endowment_min=25, 
                 endowment_max=50, 
                 metabolism_min=1, 
                 metabolism_max=5, 
                 vision_min=1, 
                 vision_max=5):
        # Initiate width and height of sugarscape - the 50x50 grid
        self.width = width
        self.height = height
        # Initiate population attributes - we need mins and maxs because we want a heterogenous population, which is similar to real life.
        # This allows for a diverse population which allows for variance in metabolism, vision, etc.
        self.initial_population = initial_population
        self.endowment_min = endowment_min
        self.endowment_max = endowment_max
        self.metabolism_min = metabolism_min
        self.metabolism_max = metabolism_max
        self.vision_min = vision_min
        self.vision_max = vision_max

        #initiate activation schedule
        self.schedule = mesa.time.RandomActivationByType(self)
        #initiate mesa grid class
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

        # Read in landscape file from supplementary material
        sugar_distribution = np.genfromtxt('sugar-map.txt')
        spice_distribution = np.flip(sugar_distribution, 1) # Flip on x and y axis
        # print(sugar_distribution.shape)
        # print(sugar_distribution[30][15]) # Call specific cell (x,y)
        # plt.imshow(spice_distribution, origin='lower')

        agent_id = 0


        for _, x, y in self.grid.coord_iter():
            max_sugar = sugar_distribution[x,y]
            if max_sugar > 0:
                sugar = Sugar(agent_id, self, (x,y), max_sugar)
                self.grid.place_agent(sugar, (x,y))
                self.schedule.add(sugar)
                agent_id += 1

            max_spice = spice_distribution[x, y]
            if max_spice > 0:
                spice = Spice(agent_id, self, (x, y), max_spice)
                self.grid.place_agent(spice, (x, y))
                self.schedule.add(spice)
                agent_id += 1

        for i in range(self.initial_population):

            # Get each of the agent positions
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)

            # Give agents their initial endowment - Random number between min and max for heterogenous behavior
            sugar = int(self.random.uniform(self.endowment_min, self.endowment_max + 1))
            spice = int(self.random.uniform(self.endowment_min, self.endowment_max + 1))

            # Give agents initial metabolism - How much sugar and spice does each agent need - if they do not fill their metabolism, they will die
            metabolism_sugar = int(self.random.uniform(self.metabolism_min, self.metabolism_max + 1))
            metabolism_spice = int(self.random.uniform(self.metabolism_min, self.metabolism_max + 1))

            # Give agents vision
            vision = int(self.random.uniform(self.vision_min, self.vision_max + 1))

            # Create Trader object
            trader = Trader(
                agent_id, 
                self,
                (x,y),
                moore = False,
                sugar = sugar,
                spice = spice,
                metabolism_sugar = metabolism_sugar,
                metabolism_spice = metabolism_spice,
                vision = vision       
                )
            
            # Place agent
            self.grid.place_agent(trader, (x, y))
            self.schedule.add(trader)
            agent_id += 1

    def step(self):
        '''
        Unique step function that does staged activation of sugar and spice and then
        randomly activates traders
        '''
        # Step Sugar Agents
        for sugar in self.schedule.agents_by_type[Sugar].values():
            sugar.step()

        # Step Spice Agents
        for spice in self.schedule.agents_by_type[Spice].values():
            spice.step()

        # Step Trader Agents
        # We want them to be able to run out of food and die
        # To account for the agent death and removal we need to seperate the data structure
        # I like how he wanted to rename this but none of us knew endowment
        trader_shuffle = list(self.schedule.agents_by_type[Trader].values())
        self.random.shuffle(trader_shuffle)

        for agent in trader_shuffle:
            agent.move()

        self.schedule.steps += 1 # Important for data collector to track the number of steps

    def run_model(self, step_count=1000):

        for i in range(step_count):
            self.step()


In [79]:
model = SugarscapeG1mt()

model.run_model(step_count=1)

3.1622776601683795 [(27, 24)]
0.0 [(2, 49)]
0.0 [(19, 1)]
0.0 [(29, 14)]
0.0 [(47, 48)]
0.0 [(43, 15)]
0.0 [(34, 41)]
1.0 [(18, 16)]
1.0 [(40, 36)]
0.0 [(2, 49)]
0.0 [(6, 18)]
0.0 [(48, 48)]
3.605551275463989 [(45, 37)]
0.0 [(32, 24)]
0.0 [(6, 5)]
0.0 [(14, 39)]
0.0 [(4, 6)]
0.0 [(44, 15)]
0.0 [(11, 42)]
1.0 [(34, 45)]
0.0 [(21, 40)]
0.0 [(34, 36)]
1.0 [(37, 24)]
0.0 [(40, 39)]
3.0 [(46, 24)]
0.0 [(19, 33)]
0.0 [(9, 24)]
0.0 [(26, 15)]
0.0 [(41, 27)]
0.0 [(24, 19)]
0.0 [(5, 20)]
1.0 [(8, 21), (7, 20)]
0.0 [(34, 0)]
0.0 [(45, 17)]
2.23606797749979 [(38, 11)]
0.0 [(22, 7)]
2.0 [(38, 19)]
0.0 [(43, 5)]
2.0 [(15, 35)]
0.0 [(1, 10)]
0.0 [(1, 21)]
0.0 [(9, 36)]
0.0 [(4, 13)]
0.0 [(22, 30)]
0.0 [(0, 11)]
0.0 [(8, 42)]
4.0 [(36, 25)]
0.0 [(11, 15)]
4.123105625617661 [(21, 41)]
0.0 [(10, 22)]
0.0 [(49, 31)]
0.0 [(26, 1)]
3.0 [(26, 8)]
0.0 [(41, 14)]
0.0 [(10, 5)]
1.0 [(29, 47), (30, 46)]
0.0 [(27, 48)]
1.4142135623730951 [(45, 16)]
0.0 [(29, 0)]
0.0 [(11, 36)]
0.0 [(42, 13)]
3.1622776601683795 